In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from math import sqrt
import csv
import sys

In [ ]:
from sklearn.cluster import DBSCAN
from scipy.sparse import *

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from py.pyBallMapper_Bokeh import graph_GUI, read_graph_from_list

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import cm

from bokeh.models import FixedTicker, LinearColorMapper, LogColorMapper, ColorBar, BasicTicker, LogTicker
from matplotlib.colors import to_hex

In [ ]:
from bokeh.plotting import figure, show

In [ ]:
# to deal with large csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

# Let us visualize the BM plots

In [ ]:
# table with the coloring functions
coloring_df = pd.read_csv('data/digits_y.csv', sep=' ')
# R indices start from 1...
coloring_df.index = range(1, len(coloring_df)+1)

print(coloring_df.shape)

## BM on full digits data

In [ ]:
# PLOT THE BM GRAPH o

EPSILON = 50

# adj lists path
GRAPH1_PATH = 'BM_graphs/digits_X/{}_edges'.format(EPSILON)
# point covered by each node path
GRAPH1_POINTS_PATH = 'BM_graphs/digits_X/{}_points_covered_by_landmarks'.format(EPSILON)


###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_red_palette = cm.get_cmap(name='jet') # multicolor
#my_red_palette = cm.get_cmap(name='Reds') # monochrome

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          coloring_df[['label']],
                          add_points_covered=False
                          )

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI')
my_fancy_gui = graph_GUI(G1, my_red_palette, 
                         coloring_df[['label']].columns.to_list(),
                         figsize=(800, 600),
                         render_iterations=2000)

my_fancy_gui.color_by_variable('label', MIN_VALUE=0, MAX_VALUE=9)

# add a legend
num_ticks = 10
low = 0
high = 9
color_mapper = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low-0.5, high=high+0.5)

ticks = [i for i in range(low, high+1, 1)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

my_fancy_gui.plot.add_layout(color_bar, 'right')

my_fancy_gui.plot.title = 'BM plot on full digits data - EPSILON {}'.format(EPSILON)

show(my_fancy_gui.plot)

## BM on PCA (10 dimensions) digits data

In [ ]:
# PLOT THE BM GRAPH o

EPSILON = 35

# adj lists path
GRAPH1_PATH = 'BM_graphs/digits_X_PCA/{}_edges'.format(EPSILON)
# point covered by each node path
GRAPH1_POINTS_PATH = 'BM_graphs/digits_X_PCA/{}_points_covered_by_landmarks'.format(EPSILON)


###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_red_palette = cm.get_cmap(name='jet') # multicolor
#my_red_palette = cm.get_cmap(name='Reds') # monochrome

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          coloring_df[['label']],
                          add_points_covered=False
                          )

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI')
my_fancy_gui = graph_GUI(G1, my_red_palette, 
                         coloring_df[['label']].columns.to_list(),
                         figsize=(800, 600),
                         render_iterations=2000)

my_fancy_gui.color_by_variable('label', MIN_VALUE=0, MAX_VALUE=9)

# add a legend
num_ticks = 10
low = 0
high = 9
color_mapper = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low-0.5, high=high+0.5)

ticks = [i for i in range(low, high+1, 1)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

my_fancy_gui.plot.add_layout(color_bar, 'right')

my_fancy_gui.plot.title = 'BM on PCA (10 dimensions) digits data - EPSILON {}'.format(EPSILON)

show(my_fancy_gui.plot)

# MAPPER ON BM

In [ ]:
# mapper on BM using DBscan as clustering algo
# it can use scipy csr sparse matrix to speed up computations
# inputs:
#     origin_BM   ball mapper graph
#     target_pts  pandas dataframe where to pull back elements in the BM
#     EPS         radius for the DBscan algo
#     MIN_SAMPLES min number of elements in a cluster that make it a cluster and not noise
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

def mapper_on_BM(origin_BM, target_pts, EPS, MIN_SAMPLES=1, sparse=False):
    new_graph = nx.Graph()

    # creates a sparse CSR matrix
    if sparse:
        target_pts = csr_matrix(target_pts.values)
    else:
        target_pts = target_pts.values

    for node in tqdm(origin_BM.nodes):
        X = target_pts[origin_BM.nodes[node]['points covered'], :]

        db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES).fit(X)
        # create a set of unique labels
        labels = set(db.labels_) - {-1} # outliers are not clusters

        print('\n **********')
        print('node {} contains {} points'.format(node, X.shape[0]))
        print('it has been divided in {} clusters'.format(len(labels)))

        # for each cluster
        # add a new vertex to the new graph
        for cluster in labels:
            # print the number of points in the cluster
            print('\t cluster {} has size {}'.format(cluster, (db.labels_ == cluster).sum()))
            # retrives the indeces of the points covered by the cluster
            points_covered_by_cluster = np.array(origin_BM.nodes[node]['points covered'])[np.where(db.labels_
                                                                                     == cluster)].tolist()
            # creates a node
            new_graph.add_node(str(node)+'_'+str(cluster),
                               points_covered=points_covered_by_cluster)

        for neigh in [v for v in nx.neighbors(origin_BM, node) if v > node]:
            neigh_X = target_pts[origin_BM.nodes[neigh]['points covered'], :]

            neigh_db = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES).fit(neigh_X)
            neigh_labels = set(neigh_db.labels_) - {-1} # outliers are not clusters

            # add edges between clusters that belongs to neigh in the original graph
            # if they share at least one element
            for cluster in labels:
                for neigh_cluster in neigh_labels:
                    points_covered_by_cluster = np.array(origin_BM.nodes[node]['points covered'])[np.where(db.labels_
                                                                                             == cluster)].tolist()
                    points_covered_by_neigh=np.array(origin_BM.nodes[neigh]['points covered'])[np.where(neigh_db.labels_
                                                                                          == neigh_cluster)].tolist()
                    if len( set(points_covered_by_cluster)&set(points_covered_by_neigh) ) != 0:
                        new_graph.add_edge(str(node)+'_'+str(cluster), str(neigh)+'_'+str(neigh_cluster) )


    return new_graph

In [ ]:
# we will save the mapper_on_BM to disk as pickle files 
# this way we can plot them in a second moment

In [ ]:
# Read the mapper_on_BM graph from pickle

def read_graph_from_pickle(GRAPH_PATH,
                           values_df,
                           my_palette):
    # read graph 
    G = nx.read_gpickle(GRAPH_PATH)
    
    MIN_SCALE = 7
    MAX_SCALE = 20

    MAX_NODE_SIZE = 0
    for node in G.nodes:
        if len(G.nodes[node]['points_covered']) > MAX_NODE_SIZE:
            MAX_NODE_SIZE = len(G.nodes[node]['points_covered'])

    for node in G.nodes:
        G.nodes[node]['size'] = len(G.nodes[node]['points_covered'])
        # rescale the size for display
        G.nodes[node]['size rescaled'] = MAX_SCALE*G.nodes[node]['size']/MAX_NODE_SIZE + MIN_SCALE

        G.nodes[node]['color'] = my_palette(0)

        for name, avg in values_df.loc[G.nodes[node]['points_covered']].mean().iteritems():
            G.nodes[node][name] = avg

    return G

# EXAMPLE 1
## handwritten digits to their PCA rapresentation

In [ ]:
original_BM = read_graph_from_list('BM_graphs/digits_X/50_edges',
                                   'BM_graphs/digits_X/50_points_covered_by_landmarks',
                                   add_points_covered=True)

digits_PCA = pd.read_csv('data/digits_X_PCA10.csv')
print('data loaded')

print('computing mapper on BM')
print('mapping the BM into a pointcloud of shape {}'.format(digits_PCA.shape))

pullback_to_PCA = mapper_on_BM(original_BM, digits_PCA,
                               EPS=20, MIN_SAMPLES=1)

nx.write_gpickle(pullback_to_PCA, 'pullback_digits_to_PCA.gpickle')

In [ ]:
GRAPH1_PATH = 'pullback_digits_to_PCA.gpickle'

# table with the coloring functions
coloring_df = pd.read_csv('data/digits_y.csv')
coloring_df.index = range(len(coloring_df))

###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_palette = cm.get_cmap(name='jet')

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_pickle(GRAPH1_PATH, coloring_df, my_palette)

for node in G1.nodes:
    G1.nodes[node]['points covered'] = G1.nodes[node]['points_covered']
print('loaded graph with {} nodes and {} edges'.format(len(G1.nodes), len(G1.edges)))

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
my_fancy_gui = graph_GUI(G1, my_palette, coloring_df[['label']])
my_fancy_gui.color_by_variable('label')

# add a discrete colorbar
num_ticks = 10
low = 0
high = 9
color_mapper = LinearColorMapper(palette=[to_hex(my_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low-0.5, high=high+0.5)

ticks = [i for i in range(low, high+1, 1)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

my_fancy_gui.plot.add_layout(color_bar, 'right')
my_fancy_gui.plot.title= 'pullback_digits_to_PCA'

In [ ]:
# creates an html file with the graph 
# and opens it in another tab
show(my_fancy_gui.plot)

# EXAMPLE 2
## the opposite
## handwritten digits in PCA rapresentation to the full dataset

In [ ]:
original_BM = read_graph_from_list('BM_graphs/digits_X_PCA/35_edges',
                                   'BM_graphs/digits_X_PCA/35_points_covered_by_landmarks',
                                   add_points_covered=True)

digits_PCA = pd.read_csv('data/digits_X.csv')

print('data loaded')
print('computing mapper on BM')
print('mapping the BM into a pointcloud of shape {}'.format(digits_PCA.shape))
pullback_to_PCA = mapper_on_BM(original_BM, digits_PCA,
                               EPS=30, MIN_SAMPLES=1)

nx.write_gpickle(pullback_to_PCA, 'pullback_digits_PCA_to_full.gpickle')

In [ ]:
GRAPH1_PATH = 'pullback_digits_PCA_to_full.gpickle'

# table with the coloring functions
coloring_df = pd.read_csv('data/digits_y.csv')
coloring_df.index = range(len(coloring_df))

###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_palette = cm.get_cmap(name='jet')

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_pickle(GRAPH1_PATH, coloring_df, my_palette)

for node in G1.nodes:
    G1.nodes[node]['points covered'] = G1.nodes[node]['points_covered']
print('loaded graph with {} nodes and {} edges'.format(len(G1.nodes), len(G1.edges)))

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
my_fancy_gui = graph_GUI(G1, my_palette, coloring_df[['label']])
my_fancy_gui.color_by_variable('label')

# add a discrete colorbar
num_ticks = 10
low = 0
high = 9
color_mapper = LinearColorMapper(palette=[to_hex(my_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low-0.5, high=high+0.5)

ticks = [i for i in range(low, high+1, 1)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

my_fancy_gui.plot.add_layout(color_bar, 'right')
my_fancy_gui.plot.title= 'pullback_digits_PCA_to_full'

In [ ]:
# creates an html file with the graph 
# and opens it in another tab
show(my_fancy_gui.plot)